In [1]:
! pip install torch --quiet
! pip install pandas --quiet
! pip install torchvision --quiet


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split


# other imports
import pandas as pd
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import random

In [3]:
start_time = time.time()

# Using the inbuilt pandas function to read a dataframe from a pkl file
pokemon_df = pd.read_pickle(os.path.join('Data', 'pkl_data', 'pokemon100_2.pkl'))

end_time = time.time()

# Calculate the elapsed time
elapsed_time_seconds = end_time - start_time

elapsed_minutes = int(elapsed_time_seconds // 60)
elapsed_seconds = elapsed_time_seconds % 60

print("Elapsed Time to read the file:", elapsed_minutes, "minutes", "{:.2f}".format(elapsed_seconds), "seconds")


start_time = time.time()

Elapsed Time to read the file: 0 minutes 2.62 seconds


In [5]:
pokemon_num = 10

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, stride=1, padding=2)
        
        # Define batch normalization layers
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
                
        # Define max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Define fully connected layers
        self.fc1 = nn.Linear(256 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, pokemon_num)
        
        # Define dropout layer
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
    # Forward pass through the network
    
        # Convolutional layers with batch normalization, ReLU activation, and max pooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        
        # Flatten the tensor for the fully connected layers
        x = x.view(-1, 256 * 6 * 6)
        
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
net = Net().to(device)

In [7]:
# Load the model
model = Net()  # Initialize your model
model.load_state_dict(torch.load(os.path.join('Models', 'starters_100percent_1.pth')))

<All keys matched successfully>

In [10]:
pokemon_labels = ['pokemon_Charmander', 'pokemon_Squirtle', 'pokemon_Bulbasaur', 'pokemon_Pikachu']

rand_label = random.choice(pokemon_labels)

# Get a random sample from the 'tensor' column from a random pokemon
random_row = pokemon_df[pokemon_df[rand_label]].sample(n=1)
random_pokemon_sample = random_row['tensor']
to_pil = transforms.ToPILImage()
random_image = to_pil(random_pokemon_sample.item())
random_image.show()

# Test it with our cnn
model.eval()
output = model(random_pokemon_sample.item().unsqueeze(0))

# Print actual label
print('Actual label:', rand_label)

# print predicted index
print('Predicted index:', torch.argmax(output).item())
pred_label = pokemon_labels[torch.argmax(output).item()]
print('Predicted label:', pred_label)

Actual label: pokemon_Articuno
Predicted index: 5
Predicted label: pokemon_Articuno
